# Deep Learning Homework \#03
### Deep Learning Course $\in$ DSSC @ UniTS (Spring 2021)  

#### Submitted by [Emanuele Ballarin](mailto:emanuele@ballarin.cc)  

### Preliminaries:

#### Imports:

We start off by importing all the libraries, modules, classes and functions we are going to use *today*...

In [1]:
# Type hints
from typing import Union, Optional, List
from torch import Tensor

# Just to force-load MKL (if available)
import numpy as np

# Functions on floats and ints
from math import sqrt as math_sqrt

# Neural networks and friends
import torch as th
from torch.nn import Sequential, BatchNorm1d, Linear, LogSoftmax
import torch.nn.functional as F

# Bespoke Modules / Functions / Optimizers
from ebtorch.nn import Mish, mishlayer_init
from madgrad.madgrad import MADGRAD as MadGrad

# Model summarization
from torchinfo import summary

# Dataset handling for PyTorch
import os
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor, Normalize, Compose, Lambda

# Plotting the loss function / accuracy
from matplotlib import pyplot as plt

### Request 1:

Implement *L1 norm* regularization as a custom loss function;  

#### Defining the *ElasticLoss*

We define here an *elastic net regularization* loss adapter for any *PyTorch* loss function and model. The requested $L_1$ regularization loss can be easily obtained as a special case of the following.

In [2]:
def elastic_loss(output: Tensor, target: Tensor, model, base_loss_fn, l1_coeff, l2_coeff, device, **kwargs) -> Tensor:
    """
    Linearly-combined L1 and L2 penalty loss adapter. Supports any valid output vs. target loss
    function or criterion. Also known as 'elastic net' regularizer. Reduces to pure L1 (LASSO)
    or L2 (Ridge, Tichonov) regularizer for adequate choice of parameters.

    Parameters
    ----------
    output : Tensor
        The class-probability or prediction batch tensor (output of a model) for a given input batch.
    target : Tensor
        The correct class(es) or prediction(s) batch for such input batch.
    model : Any PyTorch-compatible object supporting the .parameters() method
        Model whose parameters need to be considered as L1 and/or L2 penalty of.
    base_loss_fn : Any PyTorch-compatible loss function
        Base loss function to be penalized.
    l1_coeff : Any Tensor-broadcastable
        Coefficient of the LASSO penalty.
    l2_coeff : Any Tensor-broadcastable
        Coefficient of the Ridge/Tichonov penalty.
    device : Any Torch-compatible device (usually 'cpu', 'cuda' or 'cuda<number>')
        The same device the model will be running on.

    Returns
    -------
    Tensor
        The value of the loss function.
    """

    # Compute base loss
    loss = base_loss_fn(output, target, **kwargs)

    # Compute model-parameters L1 and L2 "entrywise" (a.k.a. vector-equivalent) norms
    l1total: Tensor = th.tensor(0, dtype=th.float).to(device)
    l2total: Tensor = th.tensor(0, dtype=th.float).to(device)

    for p in model.parameters():
        l1total.add_(other=th.linalg.norm(p.flatten(), ord=1))
        l2total.add_(other=(th.linalg.norm(p.flatten(), ord=2).pow_(exponent=2)))    # Accum. squared inplace

    # Return loss
    return loss.add_(other=l1total, alpha=l1_coeff).add_(other=l2total, alpha=l2_coeff)

### Request 3 (out-of-order!):

We [have seen](https://github.com/ansuini/DSSC_DL_2021/blob/main/labs/02-sgd-training.ipynb) how to implement the *Quadratic Loss* for multinomial classification problems. Read the [paper from Demirkaya et al.](https://intra.ece.ucr.edu/~oymak/multiclass.pdf) (in which the Quadratic Loss is introduced along with its issues) and try implementing *Correct Class Quadratic Loss (CCQL)* in PyTorch as well.

#### Defining the *Correct Class Quadratic Loss (CCQL)*

We define here the *Correct Class Quadratic Loss (CCQL)* as an auto-differentiable custom *PyTorch* function.

In [3]:
def CCQLLoss(output: Tensor, target: Tensor, nclasses: int, w: Optional[Union[float, int]] = None, reduction: str = "mean") -> Tensor:
    """
    The Correct Class Quadratic Loss. It is useful to train a classification
    problem with `C` classes, when emphasis on correct-class accuracy needs to
    be tunable with respect to overall accuracy.
    The criterion has been introduced in [Demirkaya et al., 2020].

    .. [Demirkaya et al., 2020]: http://repository.bilkent.edu.tr/bitstream/handle/11693/54981/Exploring_the_role_of_loss_functions_in_multiclass_classification.pdf;jsessionid=034C8E24236741F33021DD62076126E2?sequence=1

    Parameters
    ----------
    output : Tensor
        The class-probability batch tensor (output of a model) for a given input batch.
    target : Tensor
        The correct class(es) batch for such input batch.
    nclasses : int
        The overall number of different classes for the problem.
    w : Union[float, int], optional
        The weight of the correct-class sub-loss, usually in [0, +infty]. The default, None
        sets the weight to its optimal value, given the number of classes. Also
        introduced in [Demirkaya et al., 2020].
    reduction : str, optional
        The reduction to apply over the input batch. One of 'mean' and 'sum'.
        Defaults to 'mean'.

    Returns
    -------
    Tensor
        The value of the CCQL loss, reduced over the batch.

    Raises
    ------
    RuntimeError
        In case an invalid reduction os specified.
    """

    # Validate 'reduction' argument
    if reduction != "sum" and reduction != "mean":
        raise RuntimeError("reduction must be either 'sum' or 'mean'")
    
    oh_targets = F.one_hot(target, num_classes=nclasses).float()

    lql = F.mse_loss(output, oh_targets, reduction=reduction)   # 2 * Lql; halving deferred;
                                                                # reduction over BOTH batch and classes

    # Set optimal w, if None
    if w is None:
        w = math_sqrt(nclasses - 1.0) - 1.0

    ccl = th.pow(1.0 - (output * oh_targets).sum(dim=1), 2) # target-th element of predictions via
                                                            # dot product with one-hot target, and
                                                            # squared elementwise

    # Reduce CCL
    if reduction == "mean":
        ccl = ccl.mean()
        lql.mul_(nclasses)    # undo mean-reduction over classes
    elif reduction == "sum":
        ccl = ccl.sum()

    return 0.5 * (lql + w * ccl)

### An *intermezzo*

#### Defining the *training / testing* machinery

In [4]:
# MNIST DataLoader(s) builder


def spawn_mnist_loaders(
    data_root="datasets/",
    batch_size_train=256,
    batch_size_test=512,
    cuda_accel=False,
    **kwargs
):

    os.makedirs(data_root, exist_ok=True)

    transforms = Compose(
        [
            ToTensor(),
            Normalize((0.1307,), (0.3081,)),  # usual normalization constants for MNIST
            Lambda(lambda x: th.flatten(x)),
        ]
    )

    trainset = MNIST(data_root, train=True, transform=transforms, download=True)
    testset = MNIST(data_root, train=False, transform=transforms, download=True)

    cuda_args = {}
    if cuda_accel:
        cuda_args = {"num_workers": 1, "pin_memory": True}

    trainloader = DataLoader(
        trainset, batch_size=batch_size_train, shuffle=True, **cuda_args
    )
    testloader = DataLoader(
        testset, batch_size=batch_size_test, shuffle=False, **cuda_args
    )
    tontrloader = DataLoader(   # tontr == test on train
        trainset, batch_size=batch_size_test, shuffle=False, **cuda_args
    )

    return trainloader, testloader, tontrloader

In [5]:
# Train / Test loop(s): inner part

def train_epoch(
    model, device, train_loader, loss_fn, optimizer, epoch, print_every_nep, inner_scheduler=None, quiet=False,
):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
        if inner_scheduler is not None:
            inner_scheduler.step()
        if not quiet and batch_idx % print_every_nep == 0:
            print(
                "Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                    epoch,
                    batch_idx * len(data),
                    len(train_loader.dataset),
                    100.0 * batch_idx / len(train_loader),
                    loss.item(),
                )
            )


def test(model, device, test_loader, loss_fn, quiet=False):
    model.eval()
    test_loss = 0
    correct = 0
    with th.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += loss_fn(
                output, target, reduction="sum"
            ).item()  # sum up batch loss
            pred = output.argmax(
                dim=1, keepdim=True
            )  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    ltlds = len(test_loader.dataset)

    test_loss /= ltlds
    
    if not quiet:
        print(
            "Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)".format(
                test_loss,
                correct,
                ltlds,
                100.0 * correct / ltlds,
            )
        )
    
    return test_loss, correct / ltlds

In [6]:
# Setting the device
# NOTE: The MADGRAD optimizer works only on GPU; ensure you have access to one ;)

device = th.device("cuda" if th.cuda.is_available() else "cpu")

In [7]:
# Hyperparameters & co.

minibatch_size_train: int = 128 # I know it's somehow high; I just want a little more stability
                                # i.e. I want to exploit the regularizing effect of large batch sizes
minibatch_size_test: int = 512

nrepochs = 100  # Will be stopped early!

In [8]:
# Test and Train dataloaders

train_loader, test_loader, test_on_train_loader = spawn_mnist_loaders(
    batch_size_train=minibatch_size_train,
    batch_size_test=minibatch_size_test,
    cuda_accel=bool(device == "cuda"),
)

In [9]:
# THE MODEL

# An easily-manageable, compact 3L-MLP with some added Mish flavour to it!
lsizes = [28*28, int(28*28*0.75), int(28*28*0.75), 10]

model = Sequential(

    # POST-INPUT BLOCK:
    Linear(in_features=lsizes[0], out_features=lsizes[1], bias=True),
    Mish(),

    # HIDDEN BLOCK:
    BatchNorm1d(num_features=lsizes[1], affine=True),   # It's harmless at worst
    Linear(in_features=lsizes[1], out_features=lsizes[2], bias=True),
    Mish(),

    # PRE-OUTPUT BLOCK:
    BatchNorm1d(num_features=lsizes[2], affine=True),   # It's harmless at worst
    Linear(in_features=lsizes[2], out_features=lsizes[3], bias=True),
    LogSoftmax(dim=1)

        ).to(device)

if device == "cpu":
    raise RuntimeError("The MADGRAD optimizer won't work without a GPU. You may want to use RAdam instead! ;)")

# The new kid of the post-Adam optimizers family. Looks like AdaGrad + Momentum
# Personal, anecdotal opinion: very favourable for convergence; needs some care to avoid overfitting.
optimizer = MadGrad(model.parameters(), lr=0.0002, weight_decay=10e-6)

In [10]:
# Initialize weights of the Mish-activated model in a way that fosters convergence
# (no opinion given on whether it favours also generalizability; usure about it)

for layr in model:
    mishlayer_init(layr)

#### Model summary

In [11]:
summary(model)

Layer (type:depth-idx)                   Param #
├─Linear: 1-1                            461,580
├─Mish: 1-2                              --
├─BatchNorm1d: 1-3                       1,176
├─Linear: 1-4                            346,332
├─Mish: 1-5                              --
├─BatchNorm1d: 1-6                       1,176
├─Linear: 1-7                            5,890
├─LogSoftmax: 1-8                        --
Total params: 816,154
Trainable params: 816,154
Non-trainable params: 0

### Request 2a:

Implement *early stopping* according to the $E_{\text{opt}}$ specification.

We will do so also using the just-implemented $L_{1}$ regularization loss.  

Additionally, the criterion is implemented with *risk-averse gracing*, i.e. the criterion triggers a halt in training only if a pre-set number of epochs have passed; however, such additional condition should never result in a worse model compared to the situation in which no *grace* had been applied.

In [12]:
lossfn = lambda x, y, **kwargs: elastic_loss(output=x, target=y, model=model, base_loss_fn=F.nll_loss, l1_coeff=0.001, l2_coeff=0.0, device=device, **kwargs)

# Eopt stopping (outer preparation)
grace_epoch = 5
ceopt = np.inf

for epoch in range(1, nrepochs + 1):
    print("TRAINING...")
    train_epoch(
        model, device, train_loader, lossfn, optimizer, epoch, print_every_nep=30, inner_scheduler=None, quiet=False,
    )
    print("\nON TRAINING SET:")
    _ = test(model, device, test_on_train_loader, lossfn, quiet=False)
    print("\nON TEST SET:")
    testloss, _ = test(model, device, test_loader, lossfn, quiet=False)
    print("\n\n")

    # Eopt stopping (per-epoch)
    if testloss > ceopt:
        if epoch > grace_epoch:
            break
        else:
            # Do not save the model if update is only due to epoch < grace_epoch
            ceopt = testloss
    else:
        ceopt = testloss
        # Always ave the model if testloss < ceopt
        th.save(model, "./eopt_model.pt")

# Load the best model
model = th.load("./eopt_model.pt")

TRAINING...
Train Epoch: 1 [0/60000 (0%)]	Loss: 30.704525
Train Epoch: 1 [3840/60000 (6%)]	Loss: 24.737648
Train Epoch: 1 [7680/60000 (13%)]	Loss: 20.740026
Train Epoch: 1 [11520/60000 (19%)]	Loss: 17.332554
Train Epoch: 1 [15360/60000 (26%)]	Loss: 14.461641
Train Epoch: 1 [19200/60000 (32%)]	Loss: 12.211861
Train Epoch: 1 [23040/60000 (38%)]	Loss: 10.423887
Train Epoch: 1 [26880/60000 (45%)]	Loss: 8.837670
Train Epoch: 1 [30720/60000 (51%)]	Loss: 7.701161
Train Epoch: 1 [34560/60000 (58%)]	Loss: 6.975154
Train Epoch: 1 [38400/60000 (64%)]	Loss: 6.348234
Train Epoch: 1 [42240/60000 (70%)]	Loss: 5.822669
Train Epoch: 1 [46080/60000 (77%)]	Loss: 5.340216
Train Epoch: 1 [49920/60000 (83%)]	Loss: 4.897836
Train Epoch: 1 [53760/60000 (90%)]	Loss: 4.546145
Train Epoch: 1 [57600/60000 (96%)]	Loss: 4.375381

ON TRAINING SET:
Average loss: 0.2046, Accuracy: 56742/60000 (95%)

ON TEST SET:
Average loss: 0.2150, Accuracy: 9396/10000 (94%)



TRAINING...
Train Epoch: 2 [0/60000 (0%)]	Loss: 4.08424

### Request 2b:

Implement *early stopping* in one of the other, additional specifications.

We will implement the $GL_{\alpha}$ criterion, and we will do so by also using the just-implemented $CCQL$ loss.  

Additionally, the criterion is implemented with *risk-averse gracing*, i.e. the criterion triggers a halt in training only if a pre-set number of epochs have passed; however, such additional condition should never result in a worse model compared to the situation in which no *grace* had been applied.

In [13]:
# But first, reset model and optimizer instances

def weight_reset(m):
    reset_parameters = getattr(m, "reset_parameters", None)
    if callable(reset_parameters):
        m.reset_parameters()

model.apply(weight_reset)

optimizer = MadGrad(model.parameters(), lr=0.0002, weight_decay=10e-6)

for layr in model:
    mishlayer_init(layr)

In [14]:
# Actual training

lossfn = lambda x, y, **kwargs: CCQLLoss(x, y, 10, **kwargs)

# GL-alpha stopping (outer preparation)
gl_alpha_value = 0.01
grace_epoch = 5
ceopt = np.inf

for epoch in range(1, nrepochs + 1):
    print("TRAINING...")
    train_epoch(
        model, device, train_loader, lossfn, optimizer, epoch, print_every_nep=30, inner_scheduler=None, quiet=False,
    )
    print("\nON TRAINING SET:")
    _, trainacc = test(model, device, test_on_train_loader, lossfn, quiet=False)
    print("\nON TEST SET:")
    testloss, _ = test(model, device, test_loader, lossfn, quiet=False)
    print("\n\n")

    # GL-alpha stopping (per-epoch)
    if testloss < ceopt:
        # Always set to the minimum testloss so far
        ceopt = testloss

    curr_gl = testloss/ceopt - 1.0

    if curr_gl > gl_alpha_value:
        if epoch > grace_epoch:
            break
    else:
        th.save(model, "./glalpha_model_model.pt")
    
    if trainacc == 1.0:
        # Crazily enough, this "little thing" may go ahead and overfit all of MNIST.
        # If so, acknowledge that and call it a day! Good job CCQL! :)
        #
        # P.S.: This exercise has been done AFTER the other specifically asking
        #       to overfit MNIST.
        print("Yay! \U0001F973")    # It's the Unicode for 🥳
        th.save(model, "./glalpha_model_model.pt")
        break

# Load the best model
model = th.load("./glalpha_model_model.pt")

TRAINING...
Train Epoch: 1 [0/60000 (0%)]	Loss: 68.453087
Train Epoch: 1 [3840/60000 (6%)]	Loss: 41.454453
Train Epoch: 1 [7680/60000 (13%)]	Loss: 37.893993
Train Epoch: 1 [11520/60000 (19%)]	Loss: 37.773579
Train Epoch: 1 [15360/60000 (26%)]	Loss: 37.348087
Train Epoch: 1 [19200/60000 (32%)]	Loss: 37.165184
Train Epoch: 1 [23040/60000 (38%)]	Loss: 37.153824
Train Epoch: 1 [26880/60000 (45%)]	Loss: 37.296307
Train Epoch: 1 [30720/60000 (51%)]	Loss: 37.304657
Train Epoch: 1 [34560/60000 (58%)]	Loss: 37.140083
Train Epoch: 1 [38400/60000 (64%)]	Loss: 37.007721
Train Epoch: 1 [42240/60000 (70%)]	Loss: 36.989956
Train Epoch: 1 [46080/60000 (77%)]	Loss: 37.061058
Train Epoch: 1 [49920/60000 (83%)]	Loss: 37.022251
Train Epoch: 1 [53760/60000 (90%)]	Loss: 36.977104
Train Epoch: 1 [57600/60000 (96%)]	Loss: 37.049873

ON TRAINING SET:
Average loss: 36.8515, Accuracy: 58533/60000 (98%)

ON TEST SET:
Average loss: 36.8962, Accuracy: 9691/10000 (97%)



TRAINING...
Train Epoch: 2 [0/60000 (0%)]	Lo

### Request 2:

#### Extra

The following is an implementation of an essential, but easy-to-use, class for the purpose of tracking running-metrics along so-called *epochs strips* (or *training strips*), as defined in [Prechelt, 97](https://page.mi.fu-berlin.de/prechelt/Biblio/stop_tricks1997.pdf).  

Though of no use in the preceding (no *early stopping* criterion making use of *strips* has been implemented), it is shared here tor completeness.

In [15]:
realnum = Union[float, int]

class EasyStrip:
    def __init__(self, strip_size: Optional[realnum] = None) -> None:
        if strip_size is not None:
            queue_size: int = int(strip_size)
        self.strip_size: Optional[int] = strip_size
        self.strip_list: List[float] = []
    
    def append(self, element) -> None:
        if self.strip_size is not None:
            while len(self.strip_list) >= self.strip_size:
                _ = self.strip_list.pop()
        self.strip_list.insert(0, element)  # Executes only when len(self.strip_list) < self.strip_size

    def make_empty(self) -> None:
        self.strip_list = []

    def resize(self, strip_size: Optional[realnum] = None) -> None:
        if strip_size is not None:
            strip_size: int = int(strip_size)
        self.strip_size = strip_size
    
    def reset(self) -> None:
        self.make_empty()
        self.resize(strip_size=None)